In [ ]:
from skimage.metrics import structural_similarity as compare_ssim
import cv2
import numpy as np

# Загрузка изображений
img1 = cv2.imread('left1.jpg')
img2 = cv2.imread('right1.jpg')

# Преобразование изображений в оттенки серого
img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

# Вычисление индекса структурного сходства (SSIM)
similarity_score, diff_map = compare_ssim(img1_gray, img2_gray, full=True)
print("Похожесть изображений: {:.4f}%".format(similarity_score * 100))

# Преобразование карты различий в 8-битный формат
diff_map = (diff_map * 255).astype("uint8")
diff_colored = cv2.merge([diff_map, diff_map, diff_map])

# Бинаризация карты различий
binary_diff = cv2.threshold(diff_map, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

# Поиск контуров на бинарном изображении
contours_list = cv2.findContours(binary_diff, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours_list = contours_list[0] if len(contours_list) == 2 else contours_list[1]

# Создание маски для изменений
change_mask = np.zeros(img1.shape, dtype='uint8')
img2_highlighted = img2.copy()

for contour in contours_list:
    contour_area = cv2.contourArea(contour)
    if contour_area > 40:  # Игнорирование мелких изменений
        x, y, width, height = cv2.boundingRect(contour)
        cv2.rectangle(img1, (x, y), (x + width, y + height), (0, 255, 0), 2)
        cv2.rectangle(img2, (x, y), (x + width, y + height), (0, 255, 0), 2)
        cv2.rectangle(diff_colored, (x, y), (x + width, y + height), (0, 255, 0), 2)
        cv2.drawContours(change_mask, [contour], 0, (255, 255, 255), -1)
        cv2.drawContours(img2_highlighted, [contour], 0, (0, 255, 0), -1)

# Отображение результатов
cv2.imshow('Image 1 with Changes', img1)
cv2.imshow('Image 2 with Changes', img2)
cv2.imshow('Difference Map', diff_map)
cv2.imshow('Colored Difference Map', diff_colored)
cv2.imshow('Change Mask', change_mask)
cv2.imshow('Highlighted Changes', img2_highlighted)
cv2.waitKey()


Похожесть изображений: 96.8570%
